<a href="https://colab.research.google.com/github/pravinjaju1005/pravinjaju1005/blob/main/NSE_TRI_to_Gsheet_Tableau_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites

## Installing and Importing required Libraries

In [ ]:
!pip install -q gspread
!pip install -q gspread-dataframe
!pip install -q nsepython

In [ ]:
from nsepython import index_total_returns
from datetime import datetime
import pandas as pd

## Authentications for connecting to Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# NSE TRI Data Extraction

In [ ]:
# Getting all Index Values in a List

def fetch_index_symbols(date: str):
    url = f"https://archives.nseindia.com/content/indices/ind_close_all_{date}.csv"
    try:
        df = pd.read_csv(url)
        # Extract unique index symbols
        symbols = df['Index Name'].dropna().unique().tolist()
        return symbols
    except Exception as e:
        print(f"❌ Failed to fetch for {date}: {e}")
        return []

# Example usage
date = "01012025"  # 1st Jan 2025
index_symbols = fetch_index_symbols(date)

In [ ]:
# Getting TRI Values for all the index

start_str = "2005-01-01"
# end_str   = "2025-06-27"
end_str   = datetime.now().strftime('%Y-%m-%d')

start_date_formatted = datetime.strptime(start_str, '%Y-%m-%d').strftime("%d-%b-%Y")
end_date_formatted = datetime.strptime(end_str, '%Y-%m-%d').strftime("%d-%b-%Y")

all_raw_dfs = []
for index_sym in index_symbols:
    try:
        df_single_index = index_total_returns(index_sym, start_date_formatted, end_date_formatted)
        if df_single_index is not None and not df_single_index.empty:
            all_raw_dfs.append(df_single_index)
    except Exception:
        pass

if not all_raw_dfs:
    print("No data could be fetched for any index.")
else:
    combined_raw_df = pd.concat(all_raw_dfs, ignore_index=True)

    combined_raw_df['Date'] = pd.to_datetime(combined_raw_df['Date'], errors='coerce')
    combined_raw_df['TotalReturnsIndex'] = pd.to_numeric(combined_raw_df['TotalReturnsIndex'], errors='coerce')
    combined_raw_df.dropna(subset=['Date', 'TotalReturnsIndex'], inplace=True)

    pivoted_df = combined_raw_df.pivot_table(index='Date', columns='Index Name', values='TotalReturnsIndex')
    pivoted_df.sort_index(ascending=False, inplace=True)

In [ ]:
pivoted_df_reset = pivoted_df.reset_index()

transformed_df = pivoted_df_reset.melt(id_vars=['Date'], var_name='Index Name', value_name='TotalReturnsIndex')

# Sort by Date (ascending is usually better for Tableau line charts)
transformed_df.sort_values(by='Date', ascending=True, inplace=True)

In [ ]:
transformed_df.head()

,Date,Index Name,TotalReturnsIndex
422137,2005-01-03,Nifty500 Shariah,NaN
178009,2005-01-03,Nifty IPO,NaN
244127,2005-01-03,Nifty MidSmallcap400 Momentum Quality 100,NaN
381449,2005-01-03,Nifty50 Shariah,NaN
208525,2005-01-03,Nifty India Corporate Group Index - Tata Group...,NaN


# Extracting Data to Google Sheet

In [ ]:
# spreadsheet_name = 'NSE TRI Data 2025' # Replace with your sheet's name
# worksheet_name = 'Sheet1' # Replace with your desired worksheet name

# try:
#     sh = gc.open(spreadsheet_name)
#     worksheet = sh.worksheet(worksheet_name)
# except gspread.exceptions.SpreadsheetNotFound:
#     print(f"Spreadsheet '{spreadsheet_name}' not found. Please create it or check the name.")
#     # You might want to create it here if it's not found:
#     # sh = gc.create(spreadsheet_name)
#     # worksheet = sh.sheet1
# except gspread.exceptions.WorksheetNotFound:
#     print(f"Worksheet '{worksheet_name}' not found in '{spreadsheet_name}'. Creating new worksheet.")
#     worksheet = sh.add_worksheet(worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed

# worksheet.clear()
# print(f"Worksheet '{worksheet_name}' in '{spreadsheet_name}' cleared.")

# from gspread_dataframe import set_with_dataframe
# set_with_dataframe(worksheet, pivoted_df, include_index=False, include_column_header=True) # Set include_index=True if you want to export the DataFrame index

# print(f"Data exported to: https://docs.google.com/spreadsheets/d/{sh.id}/edit#gid={worksheet.id}")

In [ ]:
# Data for viewing
spreadsheet_name = 'NSE TRI Data 2025' # Replace with your sheet's name
worksheet_name = 'Sheet1' # Replace with your desired worksheet name

try:
    sh = gc.open(spreadsheet_name)
    worksheet = sh.worksheet(worksheet_name)
except gspread.exceptions.SpreadsheetNotFound:
    print(f"Spreadsheet '{spreadsheet_name}' not found. Creating a new spreadsheet.")
    sh = gc.create(spreadsheet_name)
    # When you create a new spreadsheet, it automatically has a 'Sheet1'.
    # We'll try to get it, or add if for some reason it's not named 'Sheet1' or we want a different name.
    try:
        worksheet = sh.worksheet(worksheet_name)
    except gspread.exceptions.WorksheetNotFound:
        print(f"Worksheet '{worksheet_name}' not found in new spreadsheet. Creating it.")
        worksheet = sh.add_worksheet(worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed
except gspread.exceptions.WorksheetNotFound:
    print(f"Worksheet '{worksheet_name}' not found in '{spreadsheet_name}'. Creating new worksheet.")
    worksheet = sh.add_worksheet(worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed

# --- ADD THIS LINE TO CLEAR THE SHEET ---
worksheet.clear()
print(f"Worksheet '{worksheet_name}' in '{spreadsheet_name}' cleared.")
# --- END OF ADDITION ---

from gspread_dataframe import set_with_dataframe
set_with_dataframe(worksheet, pivoted_df, include_index=True, include_column_header=True) # Set include_index=True if you want to export the DataFrame index

print(f"Data exported to: https://docs.google.com/spreadsheets/d/{sh.id}/edit#gid={worksheet.id}")

Worksheet 'Sheet1' in 'NSE TRI Data 2025' cleared.
Data exported to: https://docs.google.com/spreadsheets/d/1a_o8cKfZAUwE_F_WbkMSYeGHDdkH_2E0Rj9wpNwvwrc/edit#gid=0


In [ ]:
# Data for Dashboard
spreadsheet_name = 'NSE TRI Data Dashboard' # Replace with your sheet's name
worksheet_name = 'Sheet1' # Replace with your desired worksheet name

try:
    sh = gc.open(spreadsheet_name)
    worksheet = sh.worksheet(worksheet_name)
except gspread.exceptions.SpreadsheetNotFound:
    print(f"Spreadsheet '{spreadsheet_name}' not found. Creating a new spreadsheet.")
    sh = gc.create(spreadsheet_name)
    # When you create a new spreadsheet, it automatically has a 'Sheet1'.
    # We'll try to get it, or add if for some reason it's not named 'Sheet1' or we want a different name.
    try:
        worksheet = sh.worksheet(worksheet_name)
    except gspread.exceptions.WorksheetNotFound:
        print(f"Worksheet '{worksheet_name}' not found in new spreadsheet. Creating it.")
        worksheet = sh.add_worksheet(worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed
except gspread.exceptions.WorksheetNotFound:
    print(f"Worksheet '{worksheet_name}' not found in '{spreadsheet_name}'. Creating new worksheet.")
    worksheet = sh.add_worksheet(worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed

# --- ADD THIS LINE TO CLEAR THE SHEET ---
worksheet.clear()
print(f"Worksheet '{worksheet_name}' in '{spreadsheet_name}' cleared.")
# --- END OF ADDITION ---

from gspread_dataframe import set_with_dataframe
set_with_dataframe(worksheet, transformed_df, include_index=True, include_column_header=True) # Set include_index=True if you want to export the DataFrame index

print(f"Data exported to: https://docs.google.com/spreadsheets/d/{sh.id}/edit#gid={worksheet.id}")

Spreadsheet 'NSE TRI Data Dashboard' not found. Creating a new spreadsheet.
Worksheet 'Sheet1' in 'NSE TRI Data Dashboard' cleared.
Data exported to: https://docs.google.com/spreadsheets/d/1BXM4Z04y-g9XLvMqEJOITkXK_vEfwsYaiAngVcW-X7I/edit#gid=0
